## Workflow de pruebas

In [1]:
import processing as pP
import pandas as pd
import matplotlib.pyplot as pl
import numpy as np
import confusionMatrix as cM
import decisionTree as tree
import pymongo

In [2]:
#Constants
markers = ['ESAT6', 'CFP10', 'RV1681', 'P24']

#Leer de base de datos de joinedLabApp
MONGO_URL = 'mongodb://idenmon.zapto.org:888/'

In [3]:
client = pymongo.MongoClient(MONGO_URL)
db = client.prodLaboratorio
cursor_lab_totals = db.joinedLabApp.find({})

#Crear dataframe a partir de datos lab
dfLab = pd.DataFrame(list(cursor_lab_totals))
dfLab.QR = dfLab.QR.map(lambda x: '{:.0f}'.format(x))
dfLab = dfLab.drop_duplicates('QR')
dfLab.index = dfLab['QR']
dfLab.head()

,CFP10,CFP10_Comp,CFP10_DIAG,CODIGO,Control,ESAT6,ESAT6_Comp,ESAT6_DIAG,P24,P24_Comp,...,RV1681_Comp,RV1681_DIAG,_id,age,appVersion,createdAt_App,date,gender,hour,macaddress
QR,,,,,,,,,,,,,,,,,,,,,
601170500100160,N,TN,N,101-VP,N,I,IND,N,P,TP,...,TN,N,5978fcda18c5bb9254c6e387,0.0,0.1.0,1.495233e+12,2017-05-19,Male,22:39:43,a0:39:f7:1b:af:49
601170500100147,N,TN,N,105-VP,N,N,TN,N,N,FN,...,TN,N,5978fcda18c5bb9254c6e38a,0.0,0.1.0,1.495233e+12,2017-05-19,Male,22:31:14,a0:39:f7:1b:af:49
601170500100103,P,TP,P,107-VP,N,I,IND,P,I,IND,...,TP,P,5978fcda18c5bb9254c6e38f,0.0,0.1.0,1.495132e+12,2017-05-18,Male,18:21:17,a0:39:f7:1b:af:49
601170500100157,N,TN,N,108-VP,N,N,TN,N,P,TP,...,TN,N,5978fcda18c5bb9254c6e392,0.0,0.1.0,1.495232e+12,2017-05-19,Male,22:21:19,a0:39:f7:1b:af:49
601170500100173,N,TN,N,109-VP,N,N,TN,N,P,TP,...,TN,N,5978fcda18c5bb9254c6e393,0.0,0.1.0,1.495234e+12,2017-05-19,Male,22:46:07,a0:39:f7:1b:af:49


In [4]:
columns_selected = [m + '_DIAG' for m in markers] + ['Control', 'CODIGO']
dfLab = dfLab[columns_selected]

#Define lista de qrs a usar
qrs = list(dfLab.index)

In [5]:
#Run FIND Image Process through a list of qrs
listResults = []
#Lista de controles no válidos
#qrs = ["601170500100366", "601170500100359", "601170500100373", "601170500100275", "601170500100282", "601170500100287", 
#       "601170500100273", "601170500100272", "601170500100285", "601170500100291", "601170500100290", "601170500100295", 
#       "601170500100299", "601170500100304", "601170500100305", "601170500100309", "601170500100331", "601170500100341"]
qrsCopy = []
for i,qr in enumerate(qrs):
    result = pP.completePipeline(str(qr))
    if(not isinstance(result, str)):
        listResults.append(result)
        qrsCopy.append(qr)
    else:
        print("QR: " + str(qr) + " problem: " + result)
        
dfResults = pd.DataFrame.from_records(listResults, index = qrsCopy)
dfResults

QR: 601170500100170 problem: Problem to find all tests
QR: 601170500100303 problem: Problem to find all tests
QR: 601170500100301 problem: Problem to find all tests
QR: nan problem: No records!
QR: 601170500100459 problem: Problem to find all tests


,CFP10_area,CFP10_area_color,CFP10_blobs,CFP10_result_area,CFP10_result_blobs,Control_area,Control_area_color,Control_blobs,Control_result_area,Control_result_blobs,...,P24_blobs,P24_result_area,P24_result_blobs,RV1681_area,RV1681_area_color,RV1681_blobs,RV1681_result_area,RV1681_result_blobs,blurry,tb_result_area
601170500100160,39.080209,33.54,3,N,P,14.447661,40.26,2,N,I,...,4,P,P,22.344748,35.84,2,N,I,0,N
601170500100147,29.635487,34.66,3,N,P,0.000000,39.88,0,N,N,...,4,P,P,18.408851,35.98,2,N,I,0,N
601170500100103,70.822913,31.26,4,P,P,16.095511,44.12,2,N,I,...,4,P,P,53.415398,39.10,4,P,P,0,P
601170500100157,12.228845,33.64,1,N,N,0.000000,39.84,0,N,N,...,4,P,P,22.141916,39.48,2,N,I,0,N
601170500100173,42.592127,37.70,5,N,P,34.439258,40.54,4,P,P,...,4,P,P,33.542946,38.62,4,N,P,0,N
601170500100144,31.325403,38.94,3,N,P,19.071908,41.50,2,N,I,...,4,P,P,40.779715,36.40,4,N,P,0,N
601170500100105,50.793234,39.78,4,P,P,23.073671,43.38,3,N,P,...,4,P,P,54.320475,39.24,4,P,P,1,P
601170500100132,64.048872,31.30,4,P,P,0.000000,39.82,0,N,N,...,4,P,P,53.308598,36.10,4,P,P,1,P
601170500100135,68.315921,31.08,4,P,P,35.059527,42.46,3,P,P,...,4,P,P,61.344076,36.20,4,P,P,0,P
601170500100168,18.391649,33.82,2,N,I,0.000000,40.62,0,N,N,...,4,P,P,33.273675,33.50,3,N,P,0,N


In [6]:
dfLabvsApp = dfLab.join(dfResults)
dfLabvsApp = dfLabvsApp[dfLabvsApp.ESAT6_result_blobs.notnull()]

#Marker results to uppercase
for n in markers:
    dfLabvsApp[n + '_DIAG'] = dfLabvsApp[n + '_DIAG'].str.upper()
print(dfLabvsApp.columns)
dfLabvsApp.head()

Index(['ESAT6_DIAG', 'CFP10_DIAG', 'RV1681_DIAG', 'P24_DIAG', 'Control',
       'CODIGO', 'CFP10_area', 'CFP10_area_color', 'CFP10_blobs',
       'CFP10_result_area', 'CFP10_result_blobs', 'Control_area',
       'Control_area_color', 'Control_blobs', 'Control_result_area',
       'Control_result_blobs', 'ESAT6_area', 'ESAT6_area_color', 'ESAT6_blobs',
       'ESAT6_result_area', 'ESAT6_result_blobs', 'H2_area', 'H2_area_color',
       'H2_blobs', 'H2_result_area', 'H2_result_blobs', 'P24_area',
       'P24_area_color', 'P24_blobs', 'P24_result_area', 'P24_result_blobs',
       'RV1681_area', 'RV1681_area_color', 'RV1681_blobs',
       'RV1681_result_area', 'RV1681_result_blobs', 'blurry',
       'tb_result_area'],
      dtype='object')


,ESAT6_DIAG,CFP10_DIAG,RV1681_DIAG,P24_DIAG,Control,CODIGO,CFP10_area,CFP10_area_color,CFP10_blobs,CFP10_result_area,...,P24_blobs,P24_result_area,P24_result_blobs,RV1681_area,RV1681_area_color,RV1681_blobs,RV1681_result_area,RV1681_result_blobs,blurry,tb_result_area
QR,,,,,,,,,,,,,,,,,,,,,
601170500100160,N,N,N,P,N,101-VP,39.080209,33.54,3.0,N,...,4.0,P,P,22.344748,35.84,2.0,N,I,0.0,N
601170500100147,N,N,N,P,N,105-VP,29.635487,34.66,3.0,N,...,4.0,P,P,18.408851,35.98,2.0,N,I,0.0,N
601170500100103,P,P,P,P,N,107-VP,70.822913,31.26,4.0,P,...,4.0,P,P,53.415398,39.10,4.0,P,P,0.0,P
601170500100157,N,N,N,P,N,108-VP,12.228845,33.64,1.0,N,...,4.0,P,P,22.141916,39.48,2.0,N,I,0.0,N
601170500100173,N,N,N,P,N,109-VP,42.592127,37.70,5.0,N,...,4.0,P,P,33.542946,38.62,4.0,N,P,0.0,N


In [7]:
dfLabvsApp.to_csv('../data/DataFrameMerged.csv')

In [ ]:
dfLabvsApp = pd.DataFrame.from_csv('../data/DataFrameMerged.csv')

In [ ]:
marker = markers[0] #E6, CF RV, P24

In [ ]:
#Histograma para positivos (ELISA), negativos(ELISA), indeterminados (APP) algoritmo areas
posELISA = dfLabvsApp[dfLabvsApp[marker + '_DIAG'] == 'P']
negELISA = dfLabvsApp[dfLabvsApp[marker + '_DIAG'] == 'N']
indetAPP = dfLabvsApp[dfLabvsApp[marker + '_result_blobs'] == 'I']
bins = np.linspace(0, 80, 30)
pl.hist(posELISA[marker + '_area'], bins, alpha=0.9, label='P (LAB)', color=['r'])
pl.hist(negELISA[marker + '_area'], bins, alpha=0.2, label='N (LAB)', color=['b'])
pl.hist(indetAPP[marker + '_area'], bins, alpha=0.4, label='I (APP)', color=['g'])
pl.legend(loc='upper right')
pl.ylabel('Freq')
pl.xlabel('Area')
pl.savefig('../Imagenes/HistE6Area')
pl.close()

In [ ]:
#Histograma para positivos (ELISA), negativos(ELISA), indeterminados (APP) algoritmo blobs
posELISA = dfLabvsApp[dfLabvsApp[marker + '_DIAG'] == 'P']
negELISA = dfLabvsApp[dfLabvsApp[marker + '_DIAG'] == 'N']
indetAPP = dfLabvsApp[dfLabvsApp[marker + '_result_blobs'] == 'I']
bins = np.linspace(0, 5, 6)
pl.hist(posELISA[marker + '_blobs'], bins, alpha=0.9, label='P (LAB)', color=['r'])
pl.hist(negELISA[marker + '_blobs'], bins, alpha=0.2, label='N (LAB)', color=['b'])
pl.hist(indetAPP[marker + '_blobs'], bins, alpha=0.4, label='I (APP)', color=['g'])
pl.legend(loc='upper right')
pl.ylabel('Freq')
pl.xlabel('Blobs')
pl.savefig('../Imagenes/HistE6Blobs')
pl.close()

In [ ]:
#Histogramas para marcadores positivos e indeterminados App
# Create variable with TRUE if marker for Lab is P
markerLab = dfLabvsApp[marker + '_DIAG'] == 'P'

# Create variable with TRUE if marker in app is I
markerApp = dfLabvsApp[marker + '_result_blobs'] == 'I'

E6PAreas = dfLabvsApp[markerLab]
incorrectP = E6PAreas[E6PAreas[marker + '_area'] < 43]
print(marker)
print("Number of rows: " + str(len(E6PAreas)))
if (len(E6PAreas) > 0):
    print("Median: " + str(np.median(E6PAreas[marker + '_area'])))
    print("Mean: " + str(np.mean(E6PAreas[marker + '_area'])))
    print("Percentile 15: " + str(np.percentile(E6PAreas[marker + '_area'], 15)))
    print("Incorrect classified cases(FN): " + str(len(incorrectP)))
    pl.hist(E6PAreas[marker + '_area'])
    pl.ylabel('Total tests')
    pl.xlabel('Areas')
    pl.show()

In [ ]:
#Histogramas para marcadores negativos lab e indeterminados App
print(marker)
# Create variable with TRUE if marker for Lab is N
markerLab = dfLabvsApp[marker + '_DIAG'] == 'N'

# Create variable with TRUE if marker in app is I
markerApp = dfLabvsApp[marker + '_result_blobs'] == 'I'

E6NAreas = dfLabvsApp[markerLab]
incorrectN = E6NAreas[E6NAreas[marker + '_area'] > 43]
print("Number of rows: " + str(len(E6NAreas)))
if (len(E6NAreas) > 0):
    print("Mean: " + str(np.mean(E6NAreas[marker + '_area'])))
    print("Median: " + str(np.median(E6NAreas[marker + '_area'])))
    print("Percentile 85: " + str(np.percentile(E6NAreas[marker + '_area'], 85)))
    print("Incorrect classified cases (FP): " + str(len(incorrectN)))
    print("Area zero: " + str(len(dfLabvsApp[dfLabvsApp[marker + '_area'] == 0])))
    pl.hist(dfLabvsApp[marker + '_area'])
    pl.ylabel('Total tests')
    pl.xlabel('Areas')
    pl.show()

In [ ]:
def blobxArea(df, colArea, colBlob):
    df[colBlob + '_area'] = (df[colArea] + 1) / (df[colBlob] + 1)
    return df

In [ ]:
marker = 'P24'
#Th for E6, CF, RV, P24 = 30, 34, 40, 30
def areaResult(row):
    th = 30
    area = row[marker + '_area']
    if(area > th): return 'P'
    else: return 'N'
    
dfLabvsApp = blobxArea(dfLabvsApp, marker + '_area', marker + '_blobs')
dfLabvsApp[marker + "_result_area"] = dfLabvsApp.apply(areaResult, axis=1)

In [ ]:
#Histogramas para marcadores negativos lab, blobs * area
marker = 'CF'
print(marker)
# Create variable with TRUE if marker for Lab is P
markerLab = dfLabvsApp['ELF_' + marker] == 'P'

markerPBlobAreas = dfLabvsApp[markerLab]
incorrectN = markerPBlobAreas[markerPBlobAreas[marker + '_blobs_area'] < 8]
print("Number of rows: " + str(len(markerPBlobAreas)))
if (len(markerPBlobAreas) > 0):
    print("Mean: " + str(np.mean(markerPBlobAreas[marker + '_blobs_area'])))
    print("Median: " + str(np.median(markerPBlobAreas[marker + '_blobs_area'])))
    print("Percentile 15: " + str(np.percentile(markerPBlobAreas[marker + '_blobs_area'], 15)))
    print("Incorrect classified cases: " + str(len(incorrectN)))
    pl.hist(markerPBlobAreas[marker + '_blobs_area'])
    pl.ylabel('Total tests')
    pl.xlabel('BlobsAreas')
    pl.show()

In [ ]:
#Histogramas para marcadores positivos lab, blobs * area
print(marker)
# Create variable with TRUE if marker for Lab is N
markerLab = dfLabvsApp['ELF_' + marker] == 'N'

markerNBlobAreas = dfLabvsApp[markerLab]
incorrectN = markerNBlobAreas[markerNBlobAreas[marker + '_blobs_area'] > 8]
print("Number of rows: " + str(len(markerNBlobAreas)))
if (len(markerNBlobAreas) > 0):
    print("Mean: " + str(np.mean(markerNBlobAreas[marker + '_blobs_area'])))
    print("Median: " + str(np.median(markerNBlobAreas[marker + '_blobs_area'])))
    print("Percentile 85: " + str(np.percentile(markerNBlobAreas[marker + '_blobs_area'], 85)))
    print("Incorrect classified cases: " + str(len(incorrectN)))
    pl.hist(markerNBlobAreas[marker + '_blobs_area'])
    pl.ylabel('Total tests')
    pl.xlabel('BlobsAreas')
    pl.show()

In [ ]:
dfComparativeBlobsvsAreas = dfLabvsApp[['QR', 'ELF_E6', 'ELF_CF','ELF_RV', 'ELF_P24', 'E6_result', 'CF_result', 'RV_result', 'P24_result',
                         'E6_result_area', 'CF_result_area', 'RV_result_area', 'P24_result_area']]
dfComparativeBlobsvsAreas.head()

In [ ]:
dfComparativeBlobsvsAreas.to_csv('../data/DataFrameComparative.csv')

In [ ]:
#Histograma para positivos (ELISA), negativos(ELISA), indeterminados (APP) blobs*areas
posELISA = dfLabvsApp[dfLabvsApp['ELF_' + marker] == 'P']
negELISA = dfLabvsApp[dfLabvsApp['ELF_' + marker] == 'N']
indetAPP = dfLabvsApp[dfLabvsApp[marker + '_result'] == 'I']
bins = np.linspace(0, 280, 40)
pl.hist(posELISA[marker + '_blobs_area'], bins, alpha=0.9, label='P (LAB)', color=['r'])
pl.hist(negELISA[marker + '_blobs_area'], bins, alpha=0.2, label='N (LAB)', color=['b'])
pl.hist(indetAPP[marker + '_blobs_area'], bins, alpha=0.4, label='I (APP)', color=['g'])
pl.legend(loc='upper right')
pl.ylabel('Freq')
pl.xlabel('Blobs*Area')
pl.savefig('../Imagenes/Hist' + marker + 'BlobsArea')
pl.close()

In [ ]:
print(str(cM.confusionMatrix(list(dfMerged['ELF_P24']), list(dfMerged['P24_result']))))
print(str(cM.confusionMatrix(list(dfMerged['ELF_P24']), list(dfMerged['P24_result_area']))))

In [ ]:
marker = 'P24'
#Sub DataFrame for I
# Create variable with TRUE if marker in app is I
markerApp = dfLabvsApp[marker + '_result'] == 'I'

dfIndet = dfLabvsApp[markerApp]
df2, targets = tree.encodeTarget(dfLabvsApp, "ELF_" + marker)
features = [marker + '_blobs', marker + '_area']
target = 'Target_ELF_' + marker
#fitting the decision tree with scikit-learn
dt = tree.train(df2, features, target, min_samples_split=30)
tree.visualize_tree(dt, features)